<a href="https://colab.research.google.com/github/prashansanb/Machine-Learning-in-Drought-Index-Forecasting/blob/master/Temporal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
import numpy as np
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1_2CuHRVRqhEQ4oJnjFV-ZARBd_EgXpnHhXy0uABS1ok/edit?usp=sharing')
sheet = wb.worksheet('Sheet1')
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.head()

,Year,Month,P,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13
1,1901,1,-0.677879258,30.35215597,14.92018207,31.1104335,22.98956783,6.948848039,19.92086079,-0.09,0.79,-4.068,-0.354,0.46,0.82,0.59
2,1901,2,-1.606314573,14.31723445,17.53673514,32.95169891,25.21927482,6.702995617,24.55668934,0.21,-0.12,-1.859,-0.679,0.03,0.29,0.24
3,1901,3,0.182754102,57.28662663,13.89705238,28.92947934,21.38934063,4.969124649,28.79274288,0.88,0.35,-2.311,-0.554,-0.25,0.05,0.12
4,1901,4,0.694823278,54.69795653,9.437415852,23.61383264,16.50181583,3.192626672,30.89864123,0.64,0.61,-2.182,-0.317,-0.39,-0.13,0.04
5,1901,5,-1.199399257,24.91247254,6.115192854,20.73582766,13.40158642,2.23709683,20.65034191,0.08,-0.42,-0.406,-0.333,-0.56,-0.19,-0.09


In [0]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as mx
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Sequential
from keras.optimizers import SGD
from matplotlib import pyplot
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import regularizers
from sklearn.pipeline import Pipeline

#Standardization of data
names=df.columns
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_df,columns=names)

#Splitting of data into test and training as per year
x_train = scaled_df.iloc[:1321, 3:17].values.astype(float)
y_train = scaled_df.iloc[:1321, 2]
x_test = scaled_df.iloc[1322:,3:17].values.astype(float)
y_test = scaled_df.iloc[1322:, 2]
y_train  = y_train.astype(float)
y_test  = y_test.astype(float)
batch_size =len(x_train)
#print(x_train,y_train)
#print(x_test,y_test)

Using TensorFlow backend.


# Support Vector Regression

In [0]:
param_grid = {'C': [70,80, 50,100,10], 'gamma': [0.01,1,0.1],'kernel': ['rbf'], 'epsilon': [0.06, 0.08, 0.07]}
grid = GridSearchCV(SVR(),param_grid,refit=True,verbose=2, scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)
print(grid.best_estimator_)
pred=grid.predict(x_test)


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] C=70, epsilon=0.06, gamma=0.01, kernel=rbf ......................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....... C=70, epsilon=0.06, gamma=0.01, kernel=rbf, total=   0.6s
[CV] C=70, epsilon=0.06, gamma=0.01, kernel=rbf ......................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] ....... C=70, epsilon=0.06, gamma=0.01, kernel=rbf, total=   0.5s
[CV] C=70, epsilon=0.06, gamma=0.01, kernel=rbf ......................
[CV] ....... C=70, epsilon=0.06, gamma=0.01, kernel=rbf, total=   0.5s
[CV] C=70, epsilon=0.06, gamma=0.01, kernel=rbf ......................
[CV] ....... C=70, epsilon=0.06, gamma=0.01, kernel=rbf, total=   0.5s
[CV] C=70, epsilon=0.06, gamma=0.01, kernel=rbf ......................
[CV] ....... C=70, epsilon=0.06, gamma=0.01, kernel=rbf, total=   0.5s
[CV] C=70, epsilon=0.06, gamma=1, kernel=rbf .........................
[CV] .......... C=70, epsilon=0.06, gamma=1, kernel=rbf, total=   0.1s
[CV] C=70, epsilon=0.06, gamma=1, kernel=rbf .........................
[CV] .......... C=70, epsilon=0.06, gamma=1, kernel=rbf, total=   0.1s
[CV] C=70, epsilon=0.06, gamma=1, kernel=rbf .........................
[CV] .......... C=70, epsilon=0.06, gamma=1, kernel=rbf, total=   0.1s
[CV] C=70, epsilon=0.06, gamma=1, kernel=rbf .........................
[CV] .

[Parallel(n_jobs=1)]: Done 225 out of 225 | elapsed:  1.4min finished


SVR(C=70, cache_size=200, coef0=0.0, degree=3, epsilon=0.07, gamma=0.01,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
0.9418286921995671
0.049126884946519685
0.17168171636664198
0.22164585479209775


In [0]:
print(r2_score(y_test, pred, sample_weight=None, multioutput='uniform_average')) #R_squared value
print(mean_squared_error(y_test, pred))     #MSE value
print(mean_absolute_error(y_test, pred))      #MAE value
rmse = sqrt(mean_squared_error(y_test,pred))
print(rmse)                                    #RMSE value

Feature Ablation

In [0]:
#Feature Importance using Feature Ablation for SVM
def score_mod(x_train, x_test, y_train, y_test):
  clf = SVR(gamma=0.01, kernel='rbf',C=70, cache_size=200, coef0=0.0, degree=3, epsilon=0.07)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  return mx.r2_score(y_test, y_pred)
base_score= score_mod(x_train, x_test, y_train, y_test)
scores = []
for i in range(x_train.shape[1]):
  use_column = [ndx != i for ndx in range(x_train.shape[1])]
  scores.append(score_mod(x_train[:, use_column],x_test[:, use_column],y_train,y_test))

sorted(enumerate([base_score - s for s in scores]),key=lambda ndx_score: ndx_score[1],reverse=True)[:13]

[(0, 0.19143170038972668),
 (4, 0.010798072594162211),
 (1, 0.009110396621391459),
 (2, 0.007474318538376767),
 (8, 0.002086991896683932),
 (10, 0.0016185364251991574),
 (3, 0.0010538246190059386),
 (12, 0.0008225427042315436),
 (5, 0.0007515266173478974),
 (9, 0.00048499659967193853),
 (11, -0.0002560958767742205),
 (7, -0.0004015870443718761),
 (6, -0.0033738676403505252)]

# Random Forest

In [0]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 500, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(30, 90, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
# Create the random grid for Random Search
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               }
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(x_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 16.3min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 37.5min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 41.3min finished


{'n_estimators': 1500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 64, 'bootstrap': True}


0.9240944575663463
0.06410381665707578
0.25318731535579697
0.19659742346452763


In [0]:
n_estimators = [1000]
max_features = ['auto']
max_depth = [50]
max_depth.append(None)
min_samples_split = [2]
min_samples_leaf = [2]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               }

#Grid search based on the values funnelled by random search

gsc = GridSearchCV(RandomForestRegressor(), param_grid,cv=5,scoring='neg_mean_squared_error', verbose=2,n_jobs=-1)
grid_result = gsc.fit(x_train, y_train)
best_params = grid_result.best_params_
print(best_params)
pred=gsc.predict(x_test)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   46.4s finished


{'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 1000}
0.9244366237754532
0.06381485014380689
0.2526160132371004
0.19611392728572724


ValueError: ignored

In [0]:
print(r2_score(y_test, pred, sample_weight=None, multioutput='uniform_average')) #R_squared value
print(mean_squared_error(y_test, pred))     #MSE value
rmse = sqrt(mean_squared_error(y_test,pred))
print(rmse)                                     #RMSE value
print(mean_absolute_error(y_test, pred))      #MAE value

#Feature importance:
rfr= RandomForestRegressor(best_params)
rfr.fit(x_train,y_train)
importance = rfr.feature_importances_
print('Feature: %0d, Score: %.5f' % (i,v))

## ANN - Multi Layer Perceptron

In [0]:
from sklearn.neural_network import MLPRegressor
tuned_parameters = [{'hidden_layer_sizes': [13,500,150,200],
'activation': ['relu','logistic'],
'solver':['adam','sgd'], 'alpha':[0.0001,0.01], 
'batch_size':['auto'], 'learning_rate':['invscaling'],
'learning_rate_init':[0.001], 'max_iter':[1000]}]
rgr = GridSearchCV(MLPRegressor(), tuned_parameters, cv=5, scoring='neg_root_mean_squared_error')
grid_result=rgr.fit(x_train, y_train)
best_params = grid_result.best_params_
print(best_params)
pred=rgr.predict(x_test)
pred=rgr.predict(x_test)
print(r2_score(y_test, pred, sample_weight=None, multioutput='uniform_average')) #R_squared value
print(mean_squared_error(y_test, pred))     #MSE value
rmse = sqrt(mean_squared_error(y_test,pred))
score_feature=rmse
print(rmse)                                     #RMSE value
print(mean_absolute_error(y_test, pred))      #MAE value
#Feature Importance using ablation for MLP
def score_mod(x_train, x_test, y_train, y_test):
  rgr.fit(x_train, y_train)
  y_pred = rgr.predict(x_test)
  return mx.r2_score(y_test, y_pred)
base_score= score_mod(x_train, x_test, y_train, y_test)
scores = []
for i in range(x_train.shape[1]):
  use_column = [ndx != i for ndx in range(x_train.shape[1])]
  scores.append(score_mod(x_train[:, use_column],x_test[:, use_column],y_train,y_test))
print("R2 score",base_score)
sorted(enumerate([base_score - s for s in scores]),key=lambda ndx_score: ndx_score[1],reverse=True)[:13]

In [0]:
#weights for all the nodes in the Multi Layer Perceptron 
print(weights)
print("\n")
pred= mod.predict(x_test)
print(np.sqrt(mean_squared_error(y_test,pred)))
print(r2_score(y_test, pred, sample_weight=None, multioutput='uniform_average')) #R_squared value